In [1]:
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import gc

In [2]:
data_path = '../../data/output/normalized_data_X_5p.csv'
y_path = '../../data/output/y_5p.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
data.head()

,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,68,1,12569,-8.036406,-34.978267,457.0,12021,5.0,-8.036416,-34.978215,...,0,0,0,0,1,0,0,0,1,0
1,68,1,12572,-8.036294,-34.977119,623.0,12021,8.0,-8.036304,-34.976936,...,0,0,0,0,1,0,0,0,1,0
2,68,1,12573,-8.036592,-34.975683,785.0,12021,9.0,-8.036691,-34.975559,...,0,0,0,0,1,0,0,0,1,0
3,68,1,12574,-8.037339,-34.972964,1096.0,12021,12.0,-8.037271,-34.973312,...,0,0,0,0,1,0,0,0,1,0
4,68,1,12575,-8.037850,-34.971569,1260.0,12021,0.0,-8.037850,-34.971569,...,0,0,0,0,1,0,0,0,1,0


In [3]:
data.isnull().values.any()

False

In [4]:
data.columns[data.isnull().any()]

Index([], dtype='object')

In [5]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

In [6]:
del y
del data

In [7]:
def rmse_cv(model, X_train, y_train):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5))
    return(rmse)

# function to plot the RMSE vs parameter value
def plot_rmse_param(series, param_name):
    series.plot(title = "Validation Error vs " + param_name)
    plt.xlabel(param_name)
    plt.ylabel("RMSE")
    
# function to get the best RMSE and the best parameter value of the model
def best_rmse_param(series):
    best_rmse = series.min()
    best_param = series.idxmin() 
    
    return(best_rmse, best_param)

In [9]:
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0, n_jobs=-1)

gpc.fit(train_X, train_Y)

/home/veruska/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MemoryError: Unable to allocate array with shape (55907, 55907) and data type float64